## SafeHarbor Project 

#### DATA SCRAPING

In [4]:
import time
import pandas as pd
import numpy as np
import datetime
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI() #pip install pycoingecko

In [14]:
data=cg.get_coins_markets(vs_currency='usd')
data_coins=pd.DataFrame(data,columns=['id','market_cap'])
coins=data_coins['id'].iloc[0:21]

In [45]:
# Old date for scraping
Year_start= 2021
Month_start= 12
Day_start= 31
Hour_start= 1
Minute_start= 0

# New date for scraping
Year_end= 2022
Month_end= 11 
Day_end= 8
Hour_end= 1
Minute_end= 0

In [55]:
def scrape_historical_series(coin_names):
    """ Scrape historical series on the sample of coins.

        Args:
            coin_names(list): List of coins we will use for training.

        Returns:
            Dataframe with the evolution of prices, market capitalizaiton, and total volume over time, for each respective currency.
        """

    df_ts_coins1=pd.DataFrame()

    #DATE definitions
    date_time = datetime.datetime(Year_start,Month_start,Day_start) 
    date_time_now = datetime.datetime(Year_end, Month_end, Day_end)
    unix_past=time.mktime(date_time.timetuple()) #change the date format into unix for scraping
    unix_now=time.mktime(date_time_now.timetuple())
    past=datetime.datetime(Year_start, Month_start, Day_start).strftime('%Y-%m-%d')
    now=datetime.datetime(Year_end, Month_end, Day_end).strftime('%Y-%m-%d')
    datum_range=pd.date_range(start=past,end=now, freq='D')

    #empty lists
    unix_all=[]
    coins_names=[]
    #empty dataframe
    df_all=pd.DataFrame()

    #create date variable
    for val in datum_range:
        unix_all=np.append(unix_all,time.mktime(val.timetuple()))

    for coin in pd.unique(coin_names):
        time.sleep(5)
        data=cg.get_coin_market_chart_range_by_id(id=coin,vs_currency='usd',include_market_cap='true', include_24hr_vol='true', from_timestamp=unix_past,to_timestamp=unix_now)
        prices=pd.DataFrame(data['prices'],columns=['date','prices'])
        market=pd.DataFrame(data['market_caps'],columns=['date','market_caps'])
        volume=pd.DataFrame(data['total_volumes'],columns=['date','total_vol'])
        currency_name=pd.DataFrame({'coin':np.repeat(coin,len(prices))})
        ts_coins=pd.concat([currency_name,prices,market.iloc[:,1],volume.iloc[:,1]],axis=1)

        date_all=[]

        #create date variable
        for val in ts_coins['date']:
            date_all=np.append(date_all,((datetime.datetime.fromtimestamp(int(val)/1000)).strftime('%m/%d/%y, %H:%M:%S')))
        dates=pd.to_datetime(date_all, format='%m/%d/%y, %H:%M:%S')
        
        #set date as an index
        ts_coins['dates']=dates
        ts_coins=ts_coins.set_index('dates')
        ts_coins=ts_coins.drop(columns=['date'])

        #aggreggate hourly data to daily
        ts_coins=ts_coins.groupby([pd.Grouper(freq='D'), 'coin']).mean()

        df_all=pd.concat([df_all,ts_coins])
    return df_all

In [56]:
dataset=scrape_historical_series(coins)

In [58]:
dataset.to_csv('SafeHarbor_coins.csv')